📖 Framepack Colab


| |Github|
|:--|:-:|
| 🏠 **Profile** | [![GitHub](https://img.shields.io/badge/GitHub-%23121011.svg?logo=github&logoColor=white)](https://github.com/xLegende)|
| 📘 **Repos** | [![GitHub](https://img.shields.io/badge/GitHub-%23121011.svg?logo=github&logoColor=white)](https://github.com/xLegende/framepack_colab)

In [ ]:
# @title 1. Check GPU and Runtime Type
!nvidia-smi

print("\n---")
# Check memory - FramePack needs >6GB VRAM minimum.
# Check disk space - Need >30GB for models + space for outputs/environment.
!echo "--- Disk Space ---"
!df -h /
!echo "--- Memory ---"
!free -h

print("\n✅ GPU check complete. Ensure you have an GPU assigned.")
print("⚠️ Model files (>30GB) will be downloaded.")

In [ ]:
# @title 2. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

print("\n✅ Google Drive mounted at /content/drive/MyDrive.")

Mounted at /content/drive

✅ Google Drive mounted at /content/drive/MyDrive.


In [ ]:
# @title 3. Define Google Drive Output Path

#@markdown ---
#@markdown **Specify the path within your Google Drive where generated videos should be saved:**
#@markdown (e.g., `MyDrive/FramePack_Outputs`)
#@markdown The folder will be created if it doesn't exist.
gdrive_output_folder_path = "MyDrive/FramePack_Outputs" #@param {type:"string"}
#@markdown ---

import os

google_drive_mount_point = "/content/drive"
full_gdrive_output_path = os.path.join(google_drive_mount_point, gdrive_output_folder_path)

# Create the output directory on Google Drive if it doesn't exist
os.makedirs(full_gdrive_output_path, exist_ok=True)

print(f"✅ Output directory set to: {full_gdrive_output_path}")
print("Ensure this path exists or was created successfully.")

os.environ['GDRIVE_OUTPUT_PATH'] = full_gdrive_output_path

In [ ]:
# @title 4. Clone FramePack Repository
import os
repo_dir = "FramePack"
if not os.path.exists(repo_dir):
    !git clone https://github.com/lllyasviel/FramePack.git
    print("✅ FramePack repository cloned.")
    %cd FramePack
    print(f"✅ Current directory changed to: {os.getcwd()}")
else:
    print(f"✅ Directory '{repo_dir}' already exists. Skipping clone.")
    current_dir = os.getcwd()
    if not current_dir.endswith(repo_dir):
        %cd FramePack
        print(f"✅ Current directory changed to: {os.getcwd()}")
    else:
        print(f"✅ Already in directory: {current_dir}")

In [ ]:
# @title 5. Install Dependencies
#@markdown ---
#@markdown This step installs PyTorch and
#@markdown other requirements. This can take several minutes.

#@markdown ---

# Install PyTorch matching common Colab CUDA version (12.1). Adjust if needed.
print("⏳ Installing PyTorch for CUDA 12.1...")
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 -q
print("✅ PyTorch installed.")

# Install requirements from the repository file
import os
if not os.getcwd().endswith('FramePack'):
    print("Changing directory to FramePack...")
    %cd /content/FramePack

print("⏳ Installing requirements from requirements.txt...")
!pip install -r requirements.txt -q
print("✅ Repository requirements installed.")

# Install Gradio
print("⏳ Installing Gradio...")
!pip install gradio -q
print("✅ Gradio installed.")

print("⏳ Installing xformers for potential memory savings...")
!pip install -q xformers --index-url https://download.pytorch.org/whl/cu121
print("✅ xformers installation attempted.")


print("⏳ (Optional) Installing sage-attention...")
!pip install sageattention -q
print("✅ (Optional) sage-attention installed.")

print("\n✅ All dependency installations attempted.")
print("⚠️ Check output above for any errors during installation.")

In [ ]:
# @title 6. Run FramePack Gradio Application
#@markdown ---
#@markdown This cell launches the FramePack Gradio web interface.
#@markdown - It will automatically download the models (>30GB) the first time it runs. This can take a long time.
#@markdown - A public Gradio link (like `https://....gradio.live`) will appear in the output below. Click this link to open the UI in a new tab.
#@markdown ---
import os

if not os.getcwd().endswith('FramePack'):
    print("Changing directory to FramePack...")
    %cd /content/FramePack

gdrive_output_path = os.environ.get('GDRIVE_OUTPUT_PATH', 'Google Drive Path Not Found - Check Cell 3')

print("⏳ Launching FramePack Gradio App...")
print("‼️ This will start the model download process if running for the first time (>30GB). Please be patient.")
print(f"⚠️ Remember to check for output files locally (e.g., in /content/FramePack/outputs) and move them to {gdrive_output_path} if needed (using Cell 7).")

# Run the Gradio demo script. --share creates a public link.
!python demo_gradio.py --share

print("\n🛑 Gradio app execution finished or interrupted.")
print(f"➡️ If videos were generated, check local folders (like /content/FramePack/outputs/) and consider using Cell 7 to copy desired files to your Google Drive.")

In [ ]:
# @title (Optional) 7. Manually Copy Outputs to Google Drive
#@markdown ---
#@markdown If the application saved files locally (e.g., in `/content/FramePack/outputs`),
#@markdown you can run this cell **after stopping Cell 6** to copy them to your Google Drive folder.
#@markdown ---
#@markdown **Enter the local source folder path (relative to /content/FramePack/):**
local_output_folder = "outputs" #@param {type:"string"}
#@markdown ---

import os
import shutil
import glob

base_repo_path = "/content/FramePack"
local_source_path = os.path.join(base_repo_path, local_output_folder)

target_gdrive_path = os.environ.get('GDRIVE_OUTPUT_PATH')

if not target_gdrive_path:
    print("❌ Error: Google Drive output path not found. Please re-run Cell 3.")
else:
    print(f"Attempting to copy files from: {local_source_path}")
    print(f"To Google Drive destination: {target_gdrive_path}")

    if os.path.exists(local_source_path):
        files_to_copy = glob.glob(os.path.join(local_source_path, "*"))

        if not files_to_copy:
            print("No files found in the local source folder.")
        else:
            print(f"Found {len(files_to_copy)} item(s) to potentially copy...")
            copied_count = 0
            skipped_count = 0
            for item_path in files_to_copy:
                item_name = os.path.basename(item_path)
                destination_path = os.path.join(target_gdrive_path, item_name)

                try:
                    if os.path.isfile(item_path):
                        shutil.copy2(item_path, destination_path)
                        print(f"Copied File: {item_name}")
                        copied_count += 1
                    elif os.path.isdir(item_path):
                        print(f"Skipping Directory: {item_name} (copying only files by default)")
                        skipped_count +=1
                    else:
                         print(f"Skipping Unknown Item Type: {item_name}")
                         skipped_count +=1

                except Exception as e:
                    print(f"Error copying {item_name}: {e}")
                    skipped_count +=1

            print(f"\n✅ Finished copying attempt.")
            print(f"  {copied_count} item(s) copied successfully.")
            if skipped_count > 0:
                print(f"  {skipped_count} item(s) skipped (e.g., directories, errors).")
            print(f"Check your Drive folder: {target_gdrive_path}")

    else:
        print(f"❌ Local source folder '{local_source_path}' does not exist. Cannot copy files.")
        print("Please ensure the 'local_output_folder' name is correct and that the Gradio app generated files there.")